In [7]:
import pandas as pd
import xlrd as xl
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.legend as legend

In [8]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~') 

### 1.3 주휴일 프로모션

이전 분석에서는 상시 프로모션인 'Promo2'의 데이터를 활용하여 프로모션 자체가 매장의 매출에 어떤 변화를 줄 수 있는 알아보았다. 그와 달리, 이번 장에서 분석할 'Promo'는 당일에 시행하는 프로모션인만큼 특정한 날-혹은 공휴일-이 프로모션에 미치는 영향을 직접적으로 분석할 수 있는 데이터다. 

이에 따라 state holiday와 프로모션 여부의 상관관계가 있을 것이라고 가정했다. 
각 state holiday유형별로 휴교 여부와 프로모션 여부에 따라 매출 및 고객수, 그리고 객단가를 분석한다.
오픈한 날짜의 데이터만 추출했다.

휴일에는 state holiday와 school holiday가 포함된다고 가정한다.

[Data field 용어설명]

   
    -'Store' = 매장번호
    -'Sales' = 매출정보(단위:$)
    -'Customers' = 고객 수
    -'Open' = (0 : closed, 1 : open)
    -'Sales_per_person' = 객단가(매출/고객수)
    -'State Holiday' = a : public holiday(공휴일), b : Easter holiday(부활절), c : Christmas(크리스마스), 0 : none
    -'School Holiday' = 공립학교의 휴일
    -'Promo' = 당일 프로모션 여부, 0:시행안함, 1:시행함 
    -'Promo2' = 당월 상시프로모션 0:시행안함. 1:시행함


### 1.3.1 State Holiday_a
State Holiday_a(=public holiday)일 때, School Holiday와 Promotion 따른 매출 분석

state Holiday중에 State Holiday_a 데이터를 불러온다.

In [9]:
#aholiday파일은 State Holiday 유형이 a일 때 데이터를 추출하여 1차 가공한 파일이다.

a_Holi = pd.read_csv("aholiday.csv")
a_Holi.tail()

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,Sales_per_person
61,178,4,2013-05-30,10796,962,1,a,0,11.222453
62,181,4,2013-08-15,8365,836,1,a,1,10.005981
63,183,5,2015-05-01,1295,163,1,a,0,7.944785
64,203,5,2014-10-31,4841,830,0,a,1,5.832530
65,203,4,2013-10-31,4360,805,0,a,0,5.416149


위의 데이터를 보면, state holiday_a(=public holiday)에 Promotion을 실시한 날과 실시하지 않은 날이 모두 존재한다.
state holiday_a에 school holiday인 날과 아닌 날이 모두 존재한다. 이를 변수로 사용하여 분석을 실시할 예정이다.

필요하지 않은 'Date'와 'Weekofday' 변수를 제외했다.
총 65개의 데이터가 있다.

In [10]:
a_Holi=pd.read_csv('aholiday.csv', header=0, usecols= ['SchoolHoliday','Promo','Sales','Customers','Sales_per_person'], 
                                                           engine='python')
a_Holi

,Sales,Customers,Promo,SchoolHoliday,Sales_per_person
0,2689,389,1,0,6.912596
1,2656,381,0,0,6.971129
2,5807,625,1,0,9.291200
3,5307,574,1,0,9.245645
4,5065,576,1,0,8.793403
5,5749,614,1,0,9.363192
6,5173,621,1,0,8.330113
7,6594,742,1,0,8.886792
8,3335,454,1,0,7.345815
9,3020,435,0,0,6.942529


위에서 언급한 변수들을 토대로 데이터들을 그룹화하였다.
- SchoolY = school holiday인 그룹
- SchoolN = school holiday가 아닌 그룹
- PromoY = Promotion 시행 그룹
- PromoN = Promotion 시행 하지 않은 그룹

In [11]:
#school holiday인 그룹을 SchoolY에 지정하였다.
SchoolY=a_Holi[a_Holi.SchoolHoliday == 1]

In [12]:
#school holiday가 아닌 그룹을 SchoolN에 지정하였다.
SchoolN=a_Holi[a_Holi.SchoolHoliday == 0]

In [13]:
#Promotion 시행 그룹을 PromoY에 지정하였다.
PromoY=a_Holi[a_Holi.Promo == 1]

In [14]:
#Promotion 시행 하지 않은 그룹을 PromoN에 지정하였다.
PromoN=a_Holi[a_Holi.Promo == 0]

변수들의 모든 경우의 수(2X2)를 만족시키기 위해 지정된 조건들을 맞추어 4개의 그룹을 만든다. 

이 4개의 그룹들의 공통점은 모두 state holiday일 때라는 것이다.  

- SPYY= School Holiday O Promotion O 그룹
- SPYN= School Holiday O Promotion X 그룹
- SPNY= School Holiday x Promotion O 그룹
- SPNN= School Holiday X Promotion X 그룹

In [15]:
SPYY=pd.merge(SchoolY, PromoY, how='inner')
SPYN=pd.merge(SchoolY, PromoN, how='inner')
SPNY=pd.merge(SchoolN, PromoY, how='inner')
SPNN=pd.merge(SchoolN, PromoN, how='inner')

In [16]:
SPYY.head()

,Sales,Customers,Promo,SchoolHoliday,Sales_per_person
0,6184,523,1,1,11.824092
1,4526,481,1,1,9.409563
2,8971,912,1,1,9.836623
3,5660,639,1,1,8.857590
4,7470,693,1,1,10.779221


각 그룹에 "StoreType"의 변수명으로 열을 추가했다.

In [17]:
SPYY["StoreType"] = "SPYY"
SPYN["StoreType"] = "SPYN"
SPNY["StoreType"] = "SPNY"
SPNN["StoreType"] = "SPNN"

In [18]:
#School Holiday O Promotion O 그룹
SPYY  

,Sales,Customers,Promo,SchoolHoliday,Sales_per_person,StoreType
0,6184,523,1,1,11.824092,SPYY
1,4526,481,1,1,9.409563,SPYY
2,8971,912,1,1,9.836623,SPYY
3,5660,639,1,1,8.857590,SPYY
4,7470,693,1,1,10.779221,SPYY
5,8407,744,1,1,11.299731,SPYY
6,6116,626,1,1,9.769968,SPYY
7,8365,836,1,1,10.005981,SPYY


분석을 위하여 ' aa_Holi'라는 이름으로 기존의 4가지 그룹을 모두 하나의 데이터로 병합한다.

In [19]:
aa_Holi = pd.merge(SPYY,SPYN, how='outer')

In [20]:
aa_Holi = pd.merge(aa_Holi,SPNY, how='outer')

In [21]:
aa_Holi = pd.merge(aa_Holi,SPNN, how='outer')

In [22]:
aa_Holi

,Sales,Customers,Promo,SchoolHoliday,Sales_per_person,StoreType
0,6184,523,1,1,11.824092,SPYY
1,4526,481,1,1,9.409563,SPYY
2,8971,912,1,1,9.836623,SPYY
3,5660,639,1,1,8.857590,SPYY
4,7470,693,1,1,10.779221,SPYY
5,8407,744,1,1,11.299731,SPYY
6,6116,626,1,1,9.769968,SPYY
7,8365,836,1,1,10.005981,SPYY
8,4683,530,0,1,8.835849,SPYN
9,4576,703,0,1,6.509246,SPYN


피벗테이블을 사용하면 자동으로 각 변수의 평균을 구해준다. 따라서 미리 변수명을 

    매출:평균매출 ('Sales': 'Average_of_Sales')
    고객수:평균고객수 ('Customers': 'Average_of_Customers')
    객단가:평균 객단가 ('Sales_per_person': 'Average_of_Sales_per_person')

로 변경한다.

In [23]:
aa_Holi.rename(columns={'Sales': 'Average_of_Sales', 
                      'Customers': 'Average_of_Customers', 
                     'Sales_per_person': 'Average_of_Sales_per_person'}, inplace=True)

각 매장 유형별로 피벗테이블을 만들었다. 평균 고객수, 평균 매출, 그리고 평균 객단가를 표시한다.

In [24]:
pd.pivot_table(aa_Holi, index=["StoreType","SchoolHoliday", "Promo"], 
               values=["Average_of_Customers","Average_of_Sales","Average_of_Sales_per_person"], fill_value=0)

,,,Average_of_Customers,Average_of_Sales,Average_of_Sales_per_person
StoreType,SchoolHoliday,Promo,,,
SPNN,0,0,626.142857,4370.571429,6.947457
SPNY,0,1,707.333333,6702.564103,9.328970
SPYN,1,0,657.000000,4500.800000,6.986743
SPYY,1,1,681.750000,6962.375000,10.222846


State Holiday_a일 때, School Holiday과 Promotion따른 매출 분석 결과
    
    1. 평균 고객   SPNY > SPYY > SPYN > SPNN
       평균 매출   SPYY > SPNY > SPYN > SPNN
       평균 객단가  SPYY > SPNY > SPYN > SPNN

    2. 고객수가 가장 많을 때는, 등교일이면서 프로모션 할 때다.
    3. 매출 가장 높을때는, 휴교일이면서 프로모션을 할때다.

결론)

    1. 고객, 매출, 객단가가 가장 높은 경우는 모두 프로모션을 한다.
    2. State Holiday_a일때, 프로모션을 시행하는 것이 매출에 영향을 주는 가장 큰 요인이다. 

### 1.3.2 State Holiday_b
State Holiday_b(=Easter holiday)일 때, School Holiday와 Promotion따른 매출 분석

state Holiday중에 State Holiday_b 데이터를 불러온다.

In [25]:
#bholiday파일은 State Holiday 유형이 b일 때 데이터를 추출하여 1차 가공한 파일이다.

b_Holi = pd.read_csv("bholiday.csv")

state holiday_b(=Easter holiday)에 오픈한 지점은 오직 store85과 store122 뿐 이다.
state holiday_b에서 프로모션을 한 날도 있고 안 한 날도 있다.

In [26]:
b_Holi

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Sales_per_person
0,85,1,2015-04-06,11675,1706,1,0,b,1,6.843494
1,85,5,2015-04-03,13738,1812,1,1,b,1,7.581678
2,85,1,2014-04-21,11476,1686,1,0,b,1,6.806643
3,85,5,2014-04-18,12603,1721,1,1,b,1,7.323068
4,85,1,2013-04-01,12129,1749,1,0,b,1,6.934820
5,85,5,2013-03-29,13583,1836,1,1,b,1,7.398148
6,122,1,2015-04-06,3651,485,1,0,b,1,7.527835
7,122,1,2014-04-21,3399,496,1,0,b,1,6.852823
8,122,1,2013-04-01,2919,426,1,0,b,1,6.852113


sotre 85와 store 122의 각 지점별 평균 고객수, 매출, 객단가를 구했다.
데이터의 수 자체가 부족할 뿐만 아니라 store85과 store122의 각 지점 고객수와 매출의 차이가 매우 크기 때문에 통합해서 비교분석하기에 적절하지 못하다고 판단하였다.

In [27]:
pd.pivot_table(b_Holi, index=[ "Store"], values=["Customers","Sales","Sales_per_person"], fill_value=0)

,Customers,Sales,Sales_per_person
Store,,,
85,1751.666667,12534,7.147975
122,469.000000,3323,7.077590


위에서 설명했듯이 통합비교는 적절하지 않으며, store122는 promotion을 실시하지 않았다. 그러므로 store 85데이터 중에서 프로모션 유무별 차이를 분석해 볼 것이다. 
state holiday_b에서 122번 매장의 데이터를 제외한다. 

In [28]:
b_Holi = b_Holi[:6]

In [29]:
b_Holi

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Sales_per_person
0,85,1,2015-04-06,11675,1706,1,0,b,1,6.843494
1,85,5,2015-04-03,13738,1812,1,1,b,1,7.581678
2,85,1,2014-04-21,11476,1686,1,0,b,1,6.806643
3,85,5,2014-04-18,12603,1721,1,1,b,1,7.323068
4,85,1,2013-04-01,12129,1749,1,0,b,1,6.934820
5,85,5,2013-03-29,13583,1836,1,1,b,1,7.398148


피벗테이블에서는 평균 고객수, 평균 매출, 그리고 평균 객단가를 명시할 예정이다. 따라서 state holiday_b에서 매출, 고객수, 그리고 객단가의 이름을 미리 바꿔준다.

In [30]:
b_Holi.rename(columns={'Sales': 'Average_of_Sales', 
                      'Customers': 'Average_of_Customers', 
                     'Sales_per_person': 'Average_of_Sales_per_person'}, inplace=True)
#아래 오류 아님.

C:\mini\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


프로모션을 한 날이 평균 고객수, 평균 매출, 평균 객단가 모두 높다.

In [31]:
pd.pivot_table(b_Holi, index=["Promo"], values=["Average_of_Customers","Average_of_Sales","Average_of_Sales_per_person"], fill_value=0)

,Average_of_Customers,Average_of_Sales,Average_of_Sales_per_person
Promo,,,
0,1713.666667,11760,6.861652
1,1789.666667,13308,7.434298


 
   결론)
    
    1. State Holiday_b일 때 오픈한 지점은 206매장 중 오직 store85와 122뿐이다. 부활절에는 거의 모든 지점이 운영하지 않는 
       다는 것을 알 수 있다.
    2. State Holiday_b일때, 프로모션을 시행하는 것이 고객수, 매출, 객단가를 상승시키는 요인이다. 
  

한계)
    
    데이터 수 자체가 부족하여 state holiday_b가 주는 영향에 대한 분석이 신뢰성이 낮다. 그러므로 state holiday_b에 대한 
    분석보다는 특수지점의 프로모션 운영에 대한 매출분석이 더욱 적합하므로, 이 장의 뒤인 '1.4 특수지점 프로모션 운영'에서 더 다
    루도록 할 것이다. 

### 1.3.3 State Holiday_c
State Holiday_c(=Christmas)일 때, School Holiday와 Promotion따른 매출 분석

state Holiday 중에 State Holiday_c인 경우의 데이터들을 불러온다.

In [32]:
#choliday파일은 State Holiday 유형이 c일 때 데이터를 추출하여 1차 가공한 파일이다.

c_Holi = pd.read_csv("choliday.csv")

놀랍게도 state holiday_c에 오픈한 지점은 85매장 뿐이었다. 85매장만 오픈해서 Store의 항목이 사라졌다.

In [33]:
c_Holi

,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Sales_per_person
0,5,2014-12-26,9741,1161,1,0,c,1,8.390181
1,4,2014-12-25,7612,968,1,0,c,1,7.863636
2,4,2013-12-26,8513,1097,1,0,c,1,7.760255
3,3,2013-12-25,6138,871,1,0,c,1,7.047072


"Store" 항목을 다시 만들어서 85매장임을 확실하게 만들어 주었다. 

In [34]:
c_Holi["Store"] = 85
c_Holi = c_Holi.loc[:,["Store", "DayOfWeek","Date","Sales","Customers","Promo","StateHoliday","SchoolHoliday","Sales_per_person"]] 

날짜를 보면, 크리스마스와 크리스마스 바로 다음 날이 state holiday_c이다. 
크리스마스보다는 크리스마스 다음 날이 더 매출이 높으며, 프로모션은 모두 하지 않았다.


In [35]:
c_Holi

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,Sales_per_person
0,85,5,2014-12-26,9741,1161,0,c,1,8.390181
1,85,4,2014-12-25,7612,968,0,c,1,7.863636
2,85,4,2013-12-26,8513,1097,0,c,1,7.760255
3,85,3,2013-12-25,6138,871,0,c,1,7.047072


결론) 
   
    1. State Holiday_c일 때 오픈한 지점은 206매장 중 오직 store85뿐이다. Christmas에는 거의 모든 지점이 운영하지 않는 
       다는 것을 알 수 있다.
    2. Christmas 당일보다는 그 다음날이 더 매출이 높다.
   
   
한계)
    
    state holiday_c의 데이터도 부족하여 이에 대한 분석보다는 특수지점의 프로모션 운영에 대한 매출분석이 더욱 적합하다. 
    이 장의 뒤인 '1.4 특수지점 프로모션 운영'에서 다루도록 할 것이다. 

### 1.4 특수지점(store85)의 프로모션

store85는 다른 매장과는 달리 state holiday_b와 state holiday_c에 매장을 운영하는 특수한 지점이다. store85는 데이터기간인 31개월동안(2013.0101.~2015.07.31) 단 한 번도 휴업한 적이 없다. 앞서 분석에서 공휴일에 따라 206개의 매장들의 31개월 기간동안 오픈했을 때의 평균을 구했다면, 이번에는 store85의 전일 데이터로 분석을 할 것이다. 

#### 1.4.1 State Holiday_b와 c

피벗테이블에서는 평균 고객수, 평균 매출, 그리고 평균 객단가를 명시할 예정이다.따라서 state holiday_b와 state_holiday_c에서 매출, 고객수, 그리고 객단가의 이름을 미리 바꿔준다.

In [36]:
b_Holi.rename(columns={'Sales': 'Average_of_Sales', 
                      'Customers': 'Average_of_Customers', 
                     'Sales_per_person': 'Average_of_Sales_per_person'}, inplace=True)
#아래 오류 아님.

C:\mini\lib\site-packages\pandas\core\frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [37]:
c_Holi.rename(columns={'Sales': 'Average_of_Sales', 
                      'Customers': 'Average_of_Customers', 
                     'Sales_per_person': 'Average_of_Sales_per_person'}, inplace=True)

In [38]:
b_Holi1=b_Holi[["Average_of_Customers", "Average_of_Sales", "Average_of_Sales_per_person"]]
c_Holi1=c_Holi[["Average_of_Customers", "Average_of_Sales", "Average_of_Sales_per_person"]]

state holiday_b와 state holiday_c의 평균 고객수, 평균 매출, 평균 객단가를 구했다. 

두 공휴일의  평균 고객수와 매출의 차이가 크지만 이 둘의 비교로는 유의미한 데이터를 추출하기에는 부적절해 보인다.

상관관계를 도출하기가 어렵다.

In [39]:
b_Holi1.aggregate(['mean'])

,Average_of_Customers,Average_of_Sales,Average_of_Sales_per_person
mean,1751.666667,12534.0,7.147975


In [40]:
c_Holi1.aggregate(['mean'])

,Average_of_Customers,Average_of_Sales,Average_of_Sales_per_person
mean,1024.25,8001.0,7.765286


#### 1.4.2 평일과의 비교

앞선 결과에서 대체적으로 '공휴일에는 프로모션을 해야한다.'는 결론이 나왔다. 
하지만 공휴일에 프로모션을 실시하지 않아도 충분한 매출이 나오는 것이 아닌가 의문이 든다. 그렇다면 매출이 저조한 평일에 하는것이 더 효율적인 프로모션전략일 수 있다. 그래서 프로모션을 하지않는 공휴일과, 프로모션하는 평일의 매출을 비교해볼 필요가 있다.   

store85는 31개월동안 매일 오픈해서 모든 데이터를 포함한다. 따라서 휴일과 프로모션을을 비교하기 용이한 데이터는 store85가 될것이다. store85의 데이터로 프로모션을 한 평일과 프로모션안한 공휴일을 비교해볼 것이다. 


초기 데이터가 매우 커서 store의 데이터를 따로 필터링해서 csv파일로 만들었다.  

In [41]:
#store85_allday파일은 store85의 전 일 데이터를 추출하여 1차 가공한 파일이다.

allday = pd.read_csv("store85_allday.csv")

In [42]:
allday.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Sales_per_person
0,85,5,2015-07-31,7791,971,1,1,0,1,8.023687
1,85,4,2015-07-30,7187,1000,1,1,0,1,7.187000
2,85,3,2015-07-29,7719,889,1,1,0,1,8.682790
3,85,2,2015-07-28,7533,920,1,1,0,1,8.188043
4,85,1,2015-07-27,7788,956,1,1,0,1,8.146444


In [43]:
#store85_PYHN파일은 store85의 프로모션을 한 평일 데이터를 추출하여 1차 가공한 파일이다.

PYHN = pd.read_csv("store85_PYHN.csv")

In [44]:
PYHN.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Sales_per_person
0,85,3,2015-07-15,7476,917,1,1,0,0,8.152672
1,85,2,2015-07-14,7626,965,1,1,0,0,7.902591
2,85,1,2015-07-13,9198,1088,1,1,0,0,8.454044
3,85,5,2015-07-03,8581,1027,1,1,0,0,8.355404
4,85,4,2015-07-02,7553,932,1,1,0,0,8.104077


    데이터 추출할 것들

    (1) 85지점이 오픈한 모든 날들의 평균 고객수, 평균 매출, 평균 객단가
    (2) 85지점의 데이터 중에서 프로모션을 안한 공휴일의 평균 고객수, 평균 매출, 평균 객단가
    (3) 85지점의 데이터 중에서 프로모션을 한 평일의 평균 고객수, 평균 매출, 평균 객단가

(1) 오픈한 모든 날의 평균 고객수/매출/객단가 구하기
- 85매장의 전일 데이터로 평균 고객수, 평균 매출, 평균 객단가를 구했다.

In [45]:
allday.rename(columns={'Sales': 'Average_of_Sales', 
                      'Customers': 'Average_of_Customers', 
                     'Sales_per_person': 'Average_of_Sales_per_person'}, inplace=True)

In [46]:
pd.pivot_table(allday, index=["Store"], 
               values=["Average_of_Sales","Average_of_Customers","Average_of_Sales_per_person"], fill_value=0)

,Average_of_Customers,Average_of_Sales,Average_of_Sales_per_person
Store,,,
85,1015.401274,7272.454352,7.119747


(2) 프로모션을 안한 공휴일의 평균 고객수/매출/객단가 구하기

 데이터들을 그룹화하였다.
- alldayProN = 오픈한 모든 일의 데이터에서 프로모션을 안한 그룹
- alldaySCH = 오픈한 모든 일 데이터에서 휴일인 그룹

오픈한 모든 일 데이터에서 휴일인 그룹, alldaySCH를 만들었다.

In [47]:
alldaySCH=allday[allday.SchoolHoliday == 1]

In [48]:
alldaySCH.head()

,Store,DayOfWeek,Date,Average_of_Sales,Average_of_Customers,Open,Promo,StateHoliday,SchoolHoliday,Average_of_Sales_per_person
0,85,5,2015-07-31,7791,971,1,1,0,1,8.023687
1,85,4,2015-07-30,7187,1000,1,1,0,1,7.187000
2,85,3,2015-07-29,7719,889,1,1,0,1,8.682790
3,85,2,2015-07-28,7533,920,1,1,0,1,8.188043
4,85,1,2015-07-27,7788,956,1,1,0,1,8.146444


오픈한 모든 일의 데이터에서 프로모션을 안한 그룹, alldayProN을 만들었다.

In [49]:
alldayProN=allday[allday.Promo == 0]

In [50]:
alldayProN.head()

,Store,DayOfWeek,Date,Average_of_Sales,Average_of_Customers,Open,Promo,StateHoliday,SchoolHoliday,Average_of_Sales_per_person
5,85,7,2015-07-26,11421,1606,1,0,0,0,7.111457
6,85,6,2015-07-25,5099,834,1,0,0,0,6.113909
7,85,5,2015-07-24,5791,831,1,0,0,1,6.968712
8,85,4,2015-07-23,5223,802,1,0,0,1,6.512469
9,85,3,2015-07-22,5504,821,1,0,0,1,6.704019


위에 만든 두 그룹의 모든 조건에 속하는 그룹으로 병합한다.
즉, 프로모션을 하지 않은 공휴일 그룹, PNHY그룹을 만들었다.

In [51]:
PNHY = pd.merge(alldaySCH,alldayProN, how='inner')

In [52]:
PNHY.head()

,Store,DayOfWeek,Date,Average_of_Sales,Average_of_Customers,Open,Promo,StateHoliday,SchoolHoliday,Average_of_Sales_per_person
0,85,5,2015-07-24,5791,831,1,0,0,1,6.968712
1,85,4,2015-07-23,5223,802,1,0,0,1,6.512469
2,85,3,2015-07-22,5504,821,1,0,0,1,6.704019
3,85,2,2015-07-21,5206,791,1,0,0,1,6.581542
4,85,1,2015-07-20,6502,935,1,0,0,1,6.954011


store85의 프로모션을 하지 않은 공휴일 그룹의 평균 고객수, 평균 매출, 평균 객단가를 구했다.

In [53]:
pd.pivot_table(PNHY, index=["Store"], 
               values=["Average_of_Sales","Average_of_Customers","Average_of_Sales_per_person"], fill_value=0)

,Average_of_Customers,Average_of_Sales,Average_of_Sales_per_person
Store,,,
85,910.103448,6222.885057,6.819831


(3) 프로모션을 한 평일의 평균 고객수/매출/객단가 구하기

- store85의 프로모션을 한 평일 그룹의 평균 고객수, 평균 매출, 평균 객단가를 구했다.

In [54]:
PYHN.rename(columns={'Sales': 'Average_of_Sales', 
                      'Customers': 'Average_of_Customers', 
                     'Sales_per_person': 'Average_of_Sales_per_person'}, inplace=True)

In [55]:
pd.pivot_table(PYHN, index=["Store"], 
               values=["Average_of_Sales","Average_of_Customers","Average_of_Sales_per_person"], fill_value=0)

,Average_of_Customers,Average_of_Sales,Average_of_Sales_per_person
Store,,,
85,989.848148,7462.811111,7.52558


위의 데이터 분석 내용을 모두 정리하자면, 아래와 같다.

    1.오픈한 모든 날들의 
    평균고객수, 평균 매출, 평균객단가는
    1015.401274, 7272.454352, 7.119747

    프로모션을 한 평일(SchoolHoliday=0, Promo=1)의 
    평균고객수, 평균 매출, 평균객단가는
    989.848148, 7462.811111, 7.52558 이다.

    프로모션을 안한 공휴일(SchoolHoliday=1, Promo=0)의
    평균고객수, 평균 매출, 평균객단가는
    910.103448, 6222.885057, 6.819831 이다.

    2.오픈한 모든 날, 프로모션을 안 한 휴일, 프로모션을 한 평일 비교
    평균고객수: 
    오픈한 모든 날들>프로모션을 한 평일>프로모션을 안한 공휴일

    평균 매출: 
    프로모션을 한 평일>오픈한 모든 날들>프로모션을 안한 공휴일

    평균객단가: 
    프로모션을 한 평일>오픈한 모든 날들>프로모션을 안한 공휴일
    
    
    3.평균고객수는 오픈한 모든 날들의 평균이 제일 높지만, 평균 매출과 평균객단가는 프로모션을 한 평일이 더 높았다.
    4.프로모션을 안한 공휴일은 평균고객수와 평균 매출과 평균객단가 모두 가장 낮다.



     결론)
 
     1. 매출에 프로모션의 여부가 공휴일 여부보다 더욱 중요하다. 
     2. 심지어 공휴일에 프로모션을 하지 않은 날은 총 평균에도 매출이 미치지 못한다. 따라서 공휴일에 오픈한 경우, 프로모션을 해
        야 한다.